In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc

In [2]:
def pr_auc_Metric(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    score = auc(recall, precision)
    return score

In [3]:
ag_roc_auc_scorer = make_scorer(
    name='roc_auc',
    score_func=pr_auc_Metric,
    optimum=1,
    greater_is_better=True,
    needs_proba=True
)

In [4]:
TIME_LIMIT = 2 * 3600

In [5]:
extra_metrics = ['roc_auc', 'accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [6]:
for i in range(0, 10):
    print(f'Fitting model {i}')
    original_data = pd.read_csv(
        f'../../data/train/set_0.csv'
        )
    synthetic_data = pd.read_csv(
        f'../../data/synthetic/tvaes/set_0.csv'
    )
    # Conteo de clases
    original_vc = original_data['ED_2Clases'].value_counts()
    synthetic_vc = synthetic_data['ED_2Clases'].value_counts()
    print(f"Original data: {original_vc['D']} {original_vc['H']}")
    print(f"Synthetic data: {synthetic_vc['D']} {synthetic_vc['H']}")

    # Calcular cuántos 'D' sintéticos tomar
    d_sample = original_vc['H'] + synthetic_vc['H'] - original_vc['D']
    print(f"Sampled data: {d_sample}")

    # Seleccionar datos sintéticos
    synth_sampled = pd.concat([
        synthetic_data[synthetic_data['ED_2Clases'] == 'H'],
        synthetic_data[synthetic_data['ED_2Clases'] == 'D'].sample(d_sample, random_state=42)
    ])

    # Combinar y mezclar
    train_data = pd.concat([original_data, synth_sampled], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Train data: {train_data['ED_2Clases'].value_counts()}")

    predictor= TabularPredictor(
        label='ED_2Clases',
        problem_type='binary',
        eval_metric=ag_roc_auc_scorer,
        sample_weight='balance_weight',
        path=f'AutogluonModels/mix/v{i}'
    )

    predictor.fit(
        train_data = train_data,
        time_limit = TIME_LIMIT,
        presets = ['high_quality'],
        auto_stack = False,
        excluded_model_types=['KNN', 'LR'],
        num_stack_levels=0,
        num_bag_folds=0,
        verbosity = 2
    )

    df_test = pd.read_csv(
        f'../../data/test/set_{i}.csv'
    )

    predictors = predictor.leaderboard(
        df_test,
        extra_metrics = extra_metrics,
        extra_info=True,
        silent=True
    )
    predictors.to_excel(
        f'../../5_results/mix/predictors_set_{i}.xlsx',
        index=False
    )

    feature_importance = predictor.feature_importance(train_data)
    feature_importance.to_excel(
        f'../../5_results/mix/feature_importance_set_{i}.xlsx',
        index=True
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       8.05 GB / 15.94 GB (50.5%)
Disk Space Avail:   57.15 GB / 446.36 GB (12.8%)
Presets specified: ['high_quality']


Fitting model 0
Original data: 223 59
Synthetic data: 323 323
Sampled data: 159
Train data: ED_2Clases
H    382
D    382
Name: count, dtype: int64


Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "c:\Users\jgala\uned\tfm\synthetic-data\4_ml\4_3_mix\AutogluonModels\mix\v0"
Train Data Rows:    764
Train Data Columns: 19
Label Column:       ED_2Clases
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = H, class 0 = D
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (H) vs negative (D) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Assigning sample weights to balance differences in frequency of classes.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    8259.13 MB
	Train Da

[1000]	valid_set's binary_logloss: 0.296458
[2000]	valid_set's binary_logloss: 0.258176
[3000]	valid_set's binary_logloss: 0.248616
[4000]	valid_set's binary_logloss: 0.249812


	0.9595	 = Validation score   (roc_auc)
	1.84s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r22 ... Training model for up to 7164.54s of the 7164.53s of remaining time.
	0.937	 = Validation score   (roc_auc)
	5.97s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost_r33 ... Training model for up to 7158.56s of the 7158.55s of remaining time.
	0.9538	 = Validation score   (roc_auc)
	0.81s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTrees_r42 ... Training model for up to 7157.72s of the 7157.71s of remaining time.
	0.9676	 = Validation score   (roc_auc)
	0.64s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost_r137 ... Training model for up to 7157.02s of the 7157.01s of remaining time.
	0.9658	 = Validation score   (roc_auc)
	1.37s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r102 ... Training model for up to 7155.64s of the 7155.63s of remainin

...

	0.9647	 = Validation score   (roc_auc)
	2.12s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForest_r39 ... Training model for up to 7112.07s of the 7112.06s of remaining time.
	0.9412	 = Validation score   (roc_auc)
	0.73s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost_r167 ... Training model for up to 7111.27s of the 7111.26s of remaining time.
	0.959	 = Validation score   (roc_auc)
	1.92s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r95 ... Training model for up to 7109.36s of the 7109.35s of remaining time.
No improvement since epoch 9: early stopping
	0.9347	 = Validation score   (roc_auc)
	1.33s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r41 ... Training model for up to 7108.00s of the 7107.99s of remaining time.
	0.9718	 = Validation score   (roc_auc)
	2.86s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: XGBoost_r98 ... Training model 

[1000]	valid_set's binary_logloss: 0.24236


	0.9639	 = Validation score   (roc_auc)
	0.74s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch_r143 ... Training model for up to 7073.89s of the 7073.87s of remaining time.
	0.9591	 = Validation score   (roc_auc)
	7.86s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: CatBoost_r128 ... Training model for up to 7066.00s of the 7065.99s of remaining time.
	0.9696	 = Validation score   (roc_auc)
	4.14s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r111 ... Training model for up to 7061.84s of the 7061.83s of remaining time.
	0.9404	 = Validation score   (roc_auc)
	2.83s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch_r31 ... Training model for up to 7058.97s of the 7058.96s of remaining time.
	0.9579	 = Validation score   (roc_auc)
	6.78s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTrees_r4 ... Training model for up to 7052.17s of the 7052.16s of 

[1000]	valid_set's binary_logloss: 0.249056


	0.96	 = Validation score   (roc_auc)
	1.1s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost_r49 ... Training model for up to 7043.41s of the 7043.40s of remaining time.
	0.9473	 = Validation score   (roc_auc)
	0.51s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: CatBoost_r5 ... Training model for up to 7042.88s of the 7042.87s of remaining time.
	0.9675	 = Validation score   (roc_auc)
	1.2s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch_r87 ... Training model for up to 7041.67s of the 7041.66s of remaining time.
	0.9726	 = Validation score   (roc_auc)
	4.9s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r71 ... Training model for up to 7036.76s of the 7036.75s of remaining time.
	0.9722	 = Validation score   (roc_auc)
	1.28s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: CatBoost_r143 ... Training model for up to 7035.46s of the 7035.45s of remaining time.
	